## Working with large-scale image data

Welcome to the session on handling large-scale image data. 

In modern microscopy, it is not uncommon to work with datasets that are gigabytes or even terabytes in size. Far too large to fit into your computer's RAM.

This notebook will introduce 2 topics related to this challenge:

1. **Dask** — a package for for lazy, parallel processing  
2. **Big image data formats** — like NGFF (OME-Zarr), and how they integrate with Dask  

### Dask

A standard TIFF file is a single, monolithic block. To access even a small part of a 100 GB TIFF, a program often needs to read it whole, which is slow, memory-intensive or even impossible.

Dask lets you work with arrays and data larger than memory via chunked, lazily evaluated operations.

You can think of it as splitting a big image into small tiles (chunks), operating on each tile, and only loading the data or performing the computation when explicitly asked to.

In [ ]:
import dask.array as da
import numpy as np

# Create a large random “image” (but lazily)
large = da.random.random((10000, 10000), chunks=(1000, 1000))

# Show its structure - this is a dask array
large


No actual data was generated yet. The `large` object is just a blueprint.

Dask stores the instructions (the "recipe") to generate each chunk. It builds a **task graph**—a step-by-step plan of what needs to be done.

We can add more instructions to this recipe. 

In [ ]:
# Simple operations, NumPy functions — sum, mean, etc.
mean = np.mean(large)

print(mean)  # This is still a Dask object, not the final number.

The calculation is only triggered when you explicitly ask for the result by calling the `.compute()` method.

When `.compute()` is called, Dask's scheduler executes the task graph. It intelligently loads only the necessary chunks into memory at any given time and runs computations in parallel on your multi-core CPU.

In [ ]:
%%time

# This is the memory-efficient way to compute a result.
# Dask calculates the mean of each chunk, then combines those results.
# The full array is never loaded into memory at once.
result = mean.compute()  # This triggers the computation
print(result)

If we call `.compute()` on the *entire Dask array*, Dask will generate all chunks and assemble the full NumPy array in memory. This will fail if the array is larger than your RAM.

In [ ]:
%%time
# Dask generates all chunks and stitches them together to one big NumPy array in memory.
# It does however utilize parallel processing
result = large.compute()
print(np.mean(result))

Dask with real image I/O

We can use a library like `dask-image` or `tifffile` library, which has special integration with Zarr, a format Dask understands natively, to read an image as Dask array.

```python
# An example using dask-image
import dask_image.imread as di
img = di.imread('../data/TEM.tiff', chunkshape=(512, 512))


```

Tifffile with `aszarr = True`

**Zarr** is a format for storing N-dimensional arrays in chunks. Instead of one giant file, a Zarr store is a directory containing many small, independent files (the chunks). A program can read just the few chunks it needs for a specific task.

The `tifffile` library can cleverly treat a standard TIFF file *as if* it were a Zarr store. This lets us open a huge TIFF lazily and wrap it in a Dask array. 

In [ ]:
import tifffile
import dask.array as da

# Lazily open a TIFF as a Zarr-backed Dask array
# This operation is instantaneous 
# because no pixel data is actually read from disk yet.
store = tifffile.imread('../data/TEM1.tiff', aszarr=True)

# Wrap that store in Dask, specifying chunk size
img = da.from_zarr(store, chunks=(500, 500))

# We can access metadata-like properties without computation.
print(f"\nShape: {img.shape}")
print(f"Data type: {img.dtype}")
print(f"Number of chunks: {img.numblocks}")
print(f"Chunk size: {img.chunksize}")

In [ ]:
print('type: ',type(img))
img

You convert a TIFF to a chunked format like Zarr for faster access in the future. This triggers Dask to read the TIFF chunk-by-chunk and write out the Zarr chunks in parallel.

In [ ]:
# Convert to Zarr and save (this will create a folder ../data/TEM.zarr)
img.to_zarr('../data/TEM.zarr', overwrite=True)

Once saved as a proper Zarr store, we can open it again lazily. This is extremely fast.

We use the `zarr` library for this.

In [ ]:
import zarr
store = zarr.open('../data/TEM.zarr', mode='r')
img = da.from_zarr(store)
img

Trying to plot the image with matplotlib.pyplot will automatically trigger .compute()!

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img, cmap='gray') # ! this will compute the entire array

Instead, you can only access a piece of data for inspection

In [ ]:
preview = img[:500, :500].compute()
plt.imshow(preview, cmap='gray')

Functions like da.mean(), da.sum(), da.std(), da.max(), and basic arithmetic (+, *, /) can be applied directly to Dask arrays.

Most functions from libraries like scikit-image, or your own custom functions are not Dask-aware. They can be used with help of `map_blocks()` or `map_overlap()`

- `map_blocks()`
    - Applies function independently to each chunk.
    - No information is shared across chunk boundaries.
- `map_overlap()`
    - Allows chunks to “borrow” extra pixels from neighboring chunks (`depth`).
    - After the operation, the extra borders are trimmed, so the output array has the same shape.

In [ ]:
from skimage.filters import gaussian
from dask.array import map_overlap, map_blocks

# Apply a Gaussian filter lazily (no computation yet)
smoothed = da.map_blocks(gaussian, img, sigma=2)

# Alternative with overlap control 
# depth=5 means 5 pixels from each side are temporarily 
# added to the chunk for computation.
smoothed_overlap = img.map_overlap(lambda arr: gaussian(arr, sigma=2), depth=5)

def smooth_image_fun(arr):
    smoothed = gaussian(arr, sigma=2)
    return smoothed
smoothed_overlap = map_overlap(smooth_image_fun, img, depth=5)

In [ ]:
# Compute for one chunk and plot it
region = smoothed_overlap[:500, :500].compute()

plt.figure(figsize=(5,5))
plt.imshow(region, cmap='gray')
plt.title("One chunk - smoothed")
plt.axis('off')
plt.show()

In [ ]:
# Running a Complex Workflow: Cellpose on a Dask Array

from cellpose import models
import dask
dask.config.set(scheduler='threads') # Use a thread-based scheduler for this workflow

# --- Setup for Cellpose ---
# 1. Initialize the Cellpose model
model = models.Cellpose(model_type='cyto3')

# --- Define a function to run Cellpose on a single chunk ---
def run_cellpose_on_chunk(image_chunk):
    # The `model.eval` function returns the mask as the first output
    mask, _, _, _ = model.eval(image_chunk, diameter=120)
    return mask

lazy_mask = da.map_overlap(
    run_cellpose_on_chunk,
    img,
    depth=5, # The size of the halo/overlap on each side
    boundary='reflect',
    dtype=np.uint16 # Cellpose masks are integer labels
)

In [ ]:
final_mask = lazy_mask.compute()
print("...Cellpose segmentation complete!")

print("Shape of the final mask:", final_mask.shape)

In [ ]:
# --- Visualize a small region of the result ---
import napari

viewer = napari.Viewer()
viewer.add_image(img, name='Original Large Image')
viewer.add_labels(final_mask, name='Dask-Cellpose Segmentation')

### Big image data formats & integration with Dask

NGFF (Next-Generation File Format) is a specification for storing large bioimaging data in a chunked, multiscale format.

OME-Zarr is an implementation of NGFF that includes standardized metadata defined by the Open Microscopy Environment (OME) on top of Zarr.

Resolution-levels (multi-scale): Zarr also supports storing multiple downsampled versions of the image. This allows a viewer like Napari to quickly display a low-resolution version when zoomed out and only load the high-resolution chunks for the specific region you zoom into.

In [ ]:
# Read the OME-Zarr file stored in S3 (online storage)

image_id = 6001247
ENPOINT_URL = 'https://uk1s3.embassy.ebi.ac.uk/'

def load_binary_from_s3(name, resolution='0'):
    root = '%s/%s/' % (name, resolution)
    return da.from_zarr(ENPOINT_URL + root)

name = 'idr/zarr/v0.1/%s.zarr' % image_id
data = load_binary_from_s3(name)
print(data.shape)

In [ ]:
data

Plot a single chunk

In [ ]:
plt.imshow(data[0,1,100,:,:].compute())

### --- Exercise ---

Work with the multi-channel OME-zarr file from above.


1. Load the OME-zarr image as dask-array (use same function), but change the resolution level (lower-resolution / level 1).
2. Show the dask-array information. (run cell with the variable name)
3. Work with the second channel only and take 20 consecutive middle slices from the z-stack
4. Create a function that takes a 3D image chunk (Z, Y, X) and performs the following steps:
    - Iterates through each 2D z-slice in the chunk.
    - Applies a Gaussian filter (`sigma=2`) to the slice.
    - Runs Cellpose `model.eval()` on the smoothed slice to get a labeled mask.
    - Return labels stack
5. Use `map_overlap() or map_blocks()` to apply your function to the sliced Dask array from step 3. You must specify parameter `dtype='int16'` for output labels
6.  Call `.compute()` on the lazy result to trigger the full workflow and get the final segmentation mask as a NumPy array.
7. Use `napari` to display the sliced image stack and overlay the computed segmentation labels.


In [ ]:
from cellpose import models
model = models.Cellpose(model_type='nuclei')

# Your code here

<details>
<summary>Click to see the example solution</summary>

```python
# EXAMPLE SOLUTION 

# 1. Load the image (lower-resolution / level 1).
res1_data = load_binary_from_s3(name, resolution=1)

# 2. Get information - run this in separate cell
res1_data

# 3. Work with the second channel only and slice every 10th z-slice
# We select T=0, C=1, and 20 Z slices.
sliced_data = res1_data[0, 1, 120:140, :, :]
print(f"Step 3: Sliced data to shape {sliced_data.shape}")

# 4. Create function for applying smoothing and Cellpose segmentation
def smooth_and_segment(arr):
    labels_stack = []
    for z_slice in arr:
        sm = gaussian(z_slice, sigma=2)
        labels, _, _, _ = model.eval(sm)
        labels_stack.append(labels)
    return np.stack(labels_stack)

# 5. Apply function on sliced array inside map_overlap() function
lazy_labels = da.map_overlap(
    smooth_and_segment,
    sliced_data,
    dtype='int16'
)

# 6. Call .compute() on the lazy result to get the final NumPy array.
final_labels = lazy_labels.compute()
print(f"Step 6: Computation complete. Final mask shape: {final_labels.shape}, dtype: {final_labels.dtype}")

# 7. Visualize sliced stack with labels with Napari.
import napari
viewer = napari.Viewer()
viewer.add_image(sliced_data.compute(), name='Sliced image')
viewer.add_labels(final_labels, name='Cellpose labels')
```